In [1]:
import requests
import random
import numpy as np

TOKEN = "pk_18e2ba4b80cb471aa96ba58fadec6b05" # edit if using your own IEX account
SYMBOLS = """abde,amd,googl,goog,adi,aapl,amat,asml,adsk,bidu,
    avgo,cdns,cern,chkp,csco,ctxs,fb,intc,intu,klac,
    lrcx,mxim,mchp,mu,msft,ntap,ntes,nvda,nxpi,qcom,
    swks,symc,snps,txn,vrsn,wdc,wday,xlnx"""
TYPES = "quote"

query = {"token": TOKEN, "symbols": SYMBOLS, "types": TYPES}
r = requests.get("https://cloud.iexapis.com/beta/stock/market/batch", params=query)
dict = r.json()
    
estimatedPrices = {}
for symbol in dict:
    estimatedPrices[symbol] = random.random() * dict[symbol]["quote"]["latestPrice"]
    + random.random() * dict[symbol]["quote"]["latestVolume"]
    + random.random() * dict[symbol]["quote"]["marketCap"]
    + random.random() * dict[symbol]["quote"]["week52High"]
    + random.random() * dict[symbol]["quote"]["week52Low"]
    + random.random() * dict[symbol]["quote"]["peRatio"]
    
arr = np.array([0, 0, 0, 0, 0, 0, 0]) # python is weird

for symbol in dict:
    latestPrice = dict[symbol]["quote"]["latestPrice"]
    latestVolume = dict[symbol]["quote"]["latestVolume"]
    marketCap = dict[symbol]["quote"]["marketCap"]
    week52High = dict[symbol]["quote"]["week52High"]
    week52Low = dict[symbol]["quote"]["week52Low"]
    peRatio = dict[symbol]["quote"]["peRatio"]
    
    arr = np.vstack([arr, [latestPrice, latestVolume, marketCap, week52High, week52Low, peRatio, estimatedPrices[symbol]]])
    
arr = arr[1::]
print(arr)


[[ 2.78900000e+01  1.27884912e+08  2.80378170e+10  3.41400000e+01
   9.04000000e+00  8.20300000e+01  1.44753110e+01]
 [ 1.23613000e+03  1.56801400e+06  8.59465119e+11  1.29144000e+03
   9.77660000e+02  2.79600000e+01  7.07090386e+02]
 [ 1.23154000e+03  1.19586600e+06  8.56273752e+11  1.27389000e+03
   9.70110000e+02  2.78600000e+01  5.77610703e+00]
 [ 1.10180000e+02  3.71685600e+06  4.05808365e+10  1.11120000e+02
   7.66200000e+01  2.59200000e+01  3.34932681e+01]
 [ 1.95090000e+02  5.09372760e+07  9.19903975e+11  2.33470000e+02
   1.42000000e+02  1.59000000e+01  6.28328353e+01]
 [ 4.13300000e+01  1.31144080e+07  3.92384127e+10  6.15700000e+01
   2.87900000e+01  1.03600000e+01  3.31067440e+01]
 [ 1.94360000e+02  7.67474000e+05  8.07159514e+10  2.21660000e+02
   1.44500000e+02  2.77300000e+01  1.11295196e+02]
 [ 1.58820000e+02  1.55002100e+06  3.47830094e+10  1.69050000e+02
   1.17720000e+02 -4.23750000e+02  1.43270741e+02]
 [ 1.71770000e+02  2.08941700e+06  4.76381628e+09  2.84220000e+0

In [36]:
'''
Genetic Algorithm for Stock Market Data
Credits to MorvanZhou: https://github.com/yuanlairucisky/MorvanZhou-Evolutionary-Algorithm
'''
class Genetic_Algorithm:


    # Get stock training data
    def __init__(self, train_data, DNA_SIZE = 10, POP_SIZE = 100, CROSS_RATE = 0.75, MUTATION_RATE = 0.01, N_GENERATIONS = 200):
        #Define constants for algorithm
        self.DNA_SIZE = DNA_SIZE # Number of input variables
        self.POP_SIZE = POP_SIZE # Number of models in current generation
        self.CROSS_RATE = CROSS_RATE # Porbability of crossover event
        self.MUTATION_RATE = MUTATION_RATE # Probability of a mutation occuring
        self.N_GENERATIONS = N_GENERATIONS # Number of generations in algorithm
        self.input_data = train_data[:,0:DNA_SIZE]
        self.output_data = train_data[:,DNA_SIZE:DNA_SIZE + 1]
        self.NUM_EXAMPLES = self.input_data.shape[0]

    # Find fitness for selection
    def get_fitness_one(self,calculated_out, real_out):
        reciporocal_reals = 1. / real_out
        averaging_factor = 1. / self.NUM_EXAMPLES
        fitness_array = np.absolute(1 - (averaging_factor * np.matmul(calculated_out, reciporocal_reals)))
        return fitness_array
    
    # Get predicted values of models
    def translateDNA(self,pop):
        transposed_input = np.transpose(self.input_data)
        return np.matmul(pop,transposed_input)

    # Get next generation based on fitness values
    def select(self,pop, fitness):
        fitness_array = np.transpose(fitness)
        selected_indexes = fitness_array.argsort()[-self.POP_SIZE:][::-1]
        selected_pop = pop[selected_indexes,:]
        return selected_pop

    # Crossover process
    def crossover(self,parent, pop):
        if np.random.rand() < self.CROSS_RATE: # Ensures crossover happens at crossover rate
            i_ = np.random.randint(0, self.POP_SIZE, size=1) # Select another individual from pop
            cross_points = np.random.randint(0, 2, size=self.DNA_SIZE).astype(np.bool) # Choose crossover points
            parent[cross_points] = pop[i_, cross_points] # Mating and produce one child
        return parent


    def mutate(self,pop): # Mutation Process
        for child in pop:
            for point in range(self.DNA_SIZE):
                if np.random.rand() < self.MUTATION_RATE: # Ensures mutation happens at mutation rate
                    if np.random.rand() < 0.5:
                        child[point] += 0.5
                    else:
                        child[point] -= 0.5
        return pop


In [37]:
pop = np.random.randint(2, size=(100,arr.shape[1] - 1)) # initialze initial population

ga = Genetic_Algorithm(arr, DNA_SIZE = arr.shape[1] - 1)

for i in range(0,ga.N_GENERATIONS):
    calculated_results = ga.translateDNA(pop)
    fitness = ga.get_fitness_one(calculated_results,ga.output_data)
    fitness_array = np.transpose(fitness)
    print("Most fit model:",pop[np.argmax(fitness),:])
    pop = ga.select(pop,fitness)
    pop = ga.mutate(pop)
    pop_copy = pop.copy()
    for i in range(0,pop.shape[0]):
        pop[i,:] = ga.crossover(pop[i,:],pop_copy)
        

Most fit model: [1 1 1 1 1 1]


IndexError: boolean index did not match indexed array along dimension 1; dimension is 100 but corresponding boolean dimension is 6